**SESSION 7. TOPIC MODELS: EVALUATION AND ANALYSIS**

Get modules ready and available:

In [1]:
import sys

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pprint import pprint

# Plotting tools
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**NOTE:** We use the text normalization function we used last time. The function is defined in Text_Normalization_Function.ipynb notebook.
We will run that notebook (to make function available) using the line below:

The Text_Normalization_Function.ipynb file should be in the same folder as the notebook you are using right now.

In [2]:
%run ./Text_Normalization_Function.ipynb

  Stored in directory: /Users/corrine/Library/Caches/pip/wheels/f5/5e/9f/dbce0d6a89f44b3f30fba0a9b1b24a288882ea2e235e515d7b
Successfully built html.parser
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python'

Make sure the text normalization function is working properly by running it on the test corpus below:

In [3]:
test_text = "<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>"
test_corpus = [test_text]
test_corpus.append(test_text)
normalized_test_corpus = normalize_corpus(test_corpus)

print("Original corpus:  ", test_corpus,"\n")
print("Processed corpus: ", normalize_corpus(test_corpus))

Original corpus:   ["<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>", "<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>"] 

Processed corpus:  ['circus dog plisse skirt jump python large foot long', 'circus dog plisse skirt jump python large foot long']


Define a function for getting keywords (words with highest weights) from the estimated topic model:

In [4]:
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

**** TOPIC MODELING: NEWS ****

The dataset here is the one we used for doing classification. 
The newspaper blogposts have 4 topics: atheism, religion, computer graphics and space sciene. 
Of course, we will not use this information for topic modeling.

Download the data and set up the data:

In [5]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
dataset = fetch_20newsgroups(shuffle=True, 
                             random_state=1, 
                             categories = categories, 
                             remove=('headers', 'footers', 'quotes'))
news_corpus = dataset.data

Normalize the corpus and create "bag-of-words" representation of the data. We'll limit the number of features to 1000. 

NOTE: It will take a couple of minutes to get the data ready! 

In [6]:
normalized_corpus_news = normalize_corpus(news_corpus)
bow_vectorizer_news = CountVectorizer()
bow_news_corpus = bow_vectorizer_news.fit_transform(normalized_corpus_news)
bow_feature_names_news = bow_vectorizer_news.get_feature_names()

Set number of topics:

In [7]:
no_topics_news = 3

Run the topic model (LDA). NOTE: It will take a couple of minutes for the estimation to finish!

In [8]:
lda_news = LatentDirichletAllocation(n_components=no_topics_news, max_iter=100,random_state = 42).fit(bow_news_corpus)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Display results:

In [9]:
no_top_words_news = 10
topic_words = get_topic_words(vectorizer = bow_vectorizer_news, 
                              lda_model = lda_news, 
                              n_words = no_top_words_news)
pd.DataFrame(topic_words, 
             columns = ["word_" + str(i) for i in range(no_top_words_news)],
             index = ["Topic_" + str(i) for i in range(len(topic_words))]) 

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
Topic_0,space,image,use,file,program,system,data,nasa,launch,edu
Topic_1,people,think,god,know,like,good,use,believe,thing,even
Topic_2,jesus,matthew,god,ra,word,christian,men,day,greek,john


Display a word vectors (words are in alphabetical order) for each topic. Each column is a topic:

In [10]:
word_weights = lda_news.components_ / lda_news.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_feature_names_news, 
                               columns = ["Topic_" + str(i) for i in range(no_topics_news)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2
000062david42,0.000013,0.000004,0.000014
000100255pixel,0.000013,0.000004,0.000014
000usd,0.000023,0.000004,0.000014
001200201pixel,0.000013,0.000004,0.000014
00index,0.000013,0.000004,0.000014
00pm,0.000022,0.000004,0.000015
01a,0.000013,0.000004,0.000014
023b,0.000025,0.000004,0.000014
04g,0.000013,0.000004,0.000014
054589e,0.000013,0.000004,0.000014


Now, sort by word weights in Topic 0 (descending order) and see the weights by 10 most frequent words in Topic 0:

In [11]:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2
space,0.010149,0.000285,0.000016
image,0.008256,0.000194,0.000018
use,0.006876,0.004251,0.000029
file,0.005271,0.000170,0.000015
program,0.005198,0.000125,0.000014
system,0.004487,0.001968,0.000015
data,0.004383,0.000068,0.000030
nasa,0.004080,0.000054,0.000014
launch,0.003944,0.000031,0.000014
edu,0.003907,0.000265,0.000847


Now, sort by word weights in Topic 1 (descending order) and see the weights by 10 most frequent words in Topic 1:

In [12]:
word_weights_df.sort_values(by='Topic_1',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2
people,0.000416,0.007939,0.000026
think,0.000479,0.007441,0.000100
god,0.000013,0.007198,0.004349
know,0.002233,0.006194,0.000448
like,0.001948,0.005216,0.000126
good,0.001758,0.004366,0.000504
use,0.006876,0.004251,0.000029
believe,0.000136,0.004238,0.000039
thing,0.000673,0.004225,0.000955
even,0.000537,0.004203,0.000212


Let's assign a dominant topic to each document in our corpus. 

To do this, we need each document be represented as a bag-of-words. 
Each word in the document is associated with some topic. Word weights in a word vector for a topic provide a measure for that association.
E.g., if you sum weights for Topic 0 across all words and their frequencies in a document, you'll get a measure of associaton of that document with Topic 0.

The attribute .transform does that for you in Python (normalized):

In [13]:
lda_news_output = lda_news.transform(bow_news_corpus)

Create a nice-looking dataframe:

In [14]:
doc_names = ["Doc_" + str(i) for i in range(len(normalized_corpus_news))]
topic_names = ["Topic_" + str(i) for i in range(no_topics_news)]
df_document_topic = pd.DataFrame(np.round(lda_news_output, 4), columns=topic_names, index=doc_names)
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
df_document_topic[0:4]

,Topic_0,Topic_1,Topic_2,dominant_topic
Doc_0,0.0004,0.4359,0.5637,2
Doc_1,0.8054,0.1928,0.0018,0
Doc_2,0.0094,0.9461,0.0445,1
Doc_3,0.9879,0.0060,0.0061,0


**** INTERACTIVE TOPIC VISUALIZATION ****

You can visualize the topics: topic size, ferquency of words in a topic versus the whole corpus, etc. You can rank words (terms) in a topic by relevancy: do you want rare and exclusive terms (i.e. found mostly in that topic) OR terms that are used frequently in that topic, not not nessesarily exclisuve to that topic?

Relevancy weight parameter is λ (0 ≤ λ ≤ 1): you can adjust it!

* small λ highlights potentially rare, but exclusive terms for the selected topic;
* large values of λ (near 1) highlight frequent, but not necessarily exclusive, terms for the selected topic;

Relevancy is measured as: 

    Relevancy = λ log[p(term | topic)] + (1 - λ) log[p(term | topic)/p(term)], 
   
   where p(term | topic) stands for term (word) weight, i.e. frequency, in a topic and p(term) stands for term's weight (frequency) in a corpus.

Additional information on how to use this visualization:
* http://www.kennyshirley.com/LDAvis/
* https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf



In [15]:
pyLDAvis.enable_notebook()
visualization_panel = pyLDAvis.sklearn.prepare(lda_news, bow_news_corpus, bow_vectorizer_news, mds='tsne')
visualization_panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0     -372.142456 -195.243271       1        1  47.154761
1     -437.769562 -443.703552       2        1  44.181980
2     -620.127258 -262.638367       3        1   8.663259, topic_info=      Category        Freq       Term       Total  loglift  logprob
term                                                               
10046  Default  315.000000      jesus  315.000000  30.0000  30.0000
17412  Default  975.000000      space  975.000000  29.0000  29.0000
7939   Default  707.000000        god  707.000000  28.0000  28.0000
9131   Default  790.000000      image  790.000000  27.0000  27.0000
13857  Default  735.000000     people  735.000000  26.0000  26.0000
18743  Default  699.000000      think  699.000000  25.0000  25.0000
14763  Default  497.000000    program  497.000000  24.0000  24.0000
7056   Default  508.000000       file  508.000000  23.0000  23.0000
3432   Default  325.000000  christian  325.000000  22.0000  22.0000
4785   Default  416.000000       data  416.000000  21.0000  21.0000
12512  Default  387.000000       nasa  387.000000  20.0000  20.0000
10671  Default  372.000000     launch  372.000000  19.0000  19.0000
20719  Default  239.000000       word  239.000000  18.0000  18.0000
2219   Default  385.000000    believe  385.000000  17.0000  17.0000
11580  Default   89.000000    matthew   89.000000  16.0000  16.0000
16421  Default  266.000000  satellite  266.000000  15.0000  15.0000
2333   Default  257.000000      bible  257.000000  14.0000  14.0000
15648  Default  278.000000   religion  278.000000  13.0000  13.0000
4818   Default  234.000000        day  234.000000  12.0000  12.0000
7321   Default  261.000000     format  261.000000  11.0000  11.0000
17311  Default  277.000000   software  277.000000  10.0000  10.0000
1748   Default  235.000000    atheist  235.000000   9.0000   9.0000
1542   Default  228.000000   argument  228.000000   8.0000   8.0000
10143  Default  243.000000       jpeg  243.000000   7.0000   7.0000
8069   Default  274.000000    graphic  274.000000   6.0000   6.0000
7494   Default  239.000000        ftp  239.000000   5.0000   5.0000
1892   Default  294.000000  available  294.000000   4.0000   4.0000
13316  Default  236.000000      orbit  236.000000   3.0000   3.0000
15158  Default   68.000000         ra   68.000000   2.0000   2.0000
5959   Default  403.000000        edu  403.000000   1.0000   1.0000
...        ...         ...        ...         ...      ...      ...
13534   Topic3   11.729114        p23   12.349231   2.3946  -7.2906
2945    Topic3   12.416426  caligiuri   13.075405   2.3944  -7.2336
15158   Topic3   63.609885         ra   68.992217   2.3649  -5.5999
8117    Topic3   44.787006      greek   48.290564   2.3708  -5.9507
10046   Topic3  214.327445      jesus  315.799908   2.0585  -4.3852
11098   Topic3   39.248123       lord   47.556481   2.2541  -6.0828
8344    Topic3   29.835062       hang   36.493700   2.2446  -6.3570
11735   Topic3   49.881663        men   67.216828   2.1478  -5.8430
14831   Topic3   33.555248   prophecy   45.647887   2.1383  -6.2395
16519   Topic3   22.758137    scholar   28.828547   2.2096  -6.6277
13715   Topic3   40.807750    passage   63.648497   2.0016  -6.0438
19912   Topic3   24.227924        van   33.087354   2.1344  -6.5652
3426    Topic3   43.024410     christ   79.550725   1.8315  -5.9909
8746    Topic3   25.706659       holy   36.259523   2.1021  -6.5059
7661    Topic3   28.354279         gd   43.496130   2.0182  -6.4079
20054   Topic3   31.365025      verse   59.112288   1.8123  -6.3070
10097   Topic3   43.771190       john  115.027046   1.4799  -5.9737
20719   Topic3   62.431594       word  239.642821   1.1010  -5.6186
17468   Topic3   35.790706      speak   84.837655   1.5830  -6.1750
4808    Topic3   36.444203      david   92.390956   1.5158  -6.1569
10402   Topic3   27.271642       king   45.954726   1.9243  -6.

**LOG-LIKELIHOOD, PERPLEXITY AND COHERENCE SCORES**

Log-likelihood, perplexity and coherence scores do not have baseline or a threshold values. 
They are used to compare and discriminate between models estimated on the same data.

We will use a function CoherenceModel() from the gensim module for computing coherence scores for our LDA topic model.

In [17]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


The function CoherenceModel() needs:
    * an array of topics in the form [["cat","dog","python"],["java","python","ruby"]], 
    * corpus with each document represented as bag-of-words, and
    * dictionary of the corpus
    
We will create those elements now using a tokenized corpus:

In [18]:
news_corpus_tokenized = [tokenize_text(normalized_corpus_news[doc_id]) for doc_id in range(len(normalized_corpus_news))]
news_dictionary = Dictionary(news_corpus_tokenized)
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

Here are the lines of Python code that would estimate an LDA topic model:

In [54]:
no_topics_news = 4
no_keywords_news = 20
lda_news = LatentDirichletAllocation(n_components=no_topics_news, max_iter=100,random_state = 42).fit(bow_news_corpus) 
topic_keywords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news, n_words=no_keywords_news)
pd.DataFrame(topic_keywords, 
             columns = ["word_" + str(i) for i in range(no_keywords_news)],
             index = ["Topic_" + str(i) for i in range(len(topic_keywords))]) 

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19
Topic_0,space,image,use,file,program,data,system,launch,nasa,edu,available,software,satellite,graphic,format,jpeg,include,ftp,orbit,information
Topic_1,think,people,like,know,god,could,use,well,good,thing,time,take,believe,even,point,way,many,give,much,post
Topic_2,p2,den,p3,p1,com,de,men,van,radius,navy,presentation,bob,edu,het,double,vice,dr,material,een,stay
Topic_3,jesus,god,christian,bible,know,people,word,matthew,day,law,even,child,point,good,man,christ,time,ra,think,many


Now let's compute the coherence score for the model:

In [56]:
cm = CoherenceModel(topics=topic_keywords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(),4))  # get coherence value

Coherence score for the model:  -3.9419


You can also see a coherence score by topic:

In [57]:
print("Coherence score by topic: ", np.round(cm.get_coherence_per_topic(),4))

Coherence score by topic:  [ -1.8577  -1.2833 -11.1032  -1.5232]


**EXERCISE 1**

Compare the coherence-scores-based evaluation for models with 2, 3, and 4 topics with your human-judgment-based evaluation of those models. What do you find? 

Coherence score for the model is -1.4506, -3.7562, -3.9419 with 2, 3, 4 topics, which doesn't agree with human-judgement-based evaluation.

In [36]:
print("Log-Likelihood (higher values are better): ", lda_news.score(bow_news_corpus))

Log-Likelihood (higher values are better):  -1661737.6873378977


Perplexity Score:

In [37]:
print("Perplexity (lower values are better): ", lda_news.perplexity(bow_news_corpus))

Perplexity (lower values are better):  4315.084570393613


**EXERCISE 2**

Compare the perplexity and log-likelihood evaluation for models with 2, 3, and 4 topics with your human-judgment-based evaluation of those models. What do you find? 

Log likelihood is -166.47k, -166.17k, -166.09k with 2, 3, and 4 topics, and perplexity is 4379, 4315, 4298 with 2, 3, and 4 topics, which agree with human-judgement-based evaluation.

**EXERCISE 3**

Write a simple script that selects the best model automatically. 
You select a criteria for "best model" (log-likelihood, perplexity, or coherence score). 
You can vary both parameter alpha and number of topics, or just number of topics.

In [60]:
perplexity = 1000000
for no_topics_news in range(2,5):
    lda_news = LatentDirichletAllocation(n_components=no_topics_news, max_iter=100,random_state = 42).fit(bow_news_corpus)
    perplexity = min(perplexity, lda_news.perplexity(bow_news_corpus))
    if perplexity==lda_news.perplexity(bow_news_corpus):
        best = lda_news
topic_keywords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = best, n_words=no_keywords_news)
pd.DataFrame(topic_keywords, 
             columns = ["word_" + str(i) for i in range(no_keywords_news)],
             index = ["Topic_" + str(i) for i in range(len(topic_keywords))]) 

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19
Topic_0,space,image,use,file,program,data,system,launch,nasa,edu,available,software,satellite,graphic,format,jpeg,include,ftp,orbit,information
Topic_1,think,people,like,know,god,could,use,well,good,thing,time,take,believe,even,point,way,many,give,much,post
Topic_2,p2,den,p3,p1,com,de,men,van,radius,navy,presentation,bob,edu,het,double,vice,dr,material,een,stay
Topic_3,jesus,god,christian,bible,know,people,word,matthew,day,law,even,child,point,good,man,christ,time,ra,think,many


In [61]:
best

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=100,
             mean_change_tol=0.001, n_components=4, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=42,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)